In [36]:
import numpy as np
import cvxpy as cp
import torch
from cvxpylayers.torch import CvxpyLayer
from pickle import load

import KKT_NN

In [37]:
x = cp.Variable(2)
point = cp.Parameter(2)
h = cp.Parameter(7)
G = cp.Parameter((7, 2))
prob = cp.Problem(cp.Minimize(cp.sum_squares(x - point)), [G @ x <= h])
generator_layer = CvxpyLayer(prob, parameters=[point, G, h], variables=[x])

In [38]:
device = torch.device("cpu" if torch.cuda.is_available() else "mps")

In [39]:
ds = load(open("dataset_loads_comp.pkl", "rb"))[:1000]

ds = np.stack([ds[i][0] for i in range(len(ds))])


In [40]:
ds.shape

(1000, 7)

In [41]:
ds = torch.tensor(ds).to(dtype=torch.float32, device=device)

In [42]:
p, q, p_pot, p_max, q_max, p_plus, q_plus = ds[..., 0], ds[..., 1], ds[..., 2], ds[..., 3], ds[..., 4], ds[..., 5], ds[..., 6]

a = torch.stack([p, q], 1)
tau1 = (q_plus - q_max)/(p_max - p_plus)
tau2 = (-q_plus + q_max)/(p_max - p_plus)
rho1 = q_max - tau1*p_plus
rho2 = -q_max - tau2*p_plus

In [43]:
G_val = torch.Tensor(
                [
                    [-1, 0], [1, 0], [1, 0], [0, -1], [0, 1], [0, 1], [0, -1]
                ]
            ).to(dtype=torch.float32, device=device).repeat(len(a), 1, 1)

In [44]:
G_val[..., -2 , 0] = tau1
G_val[..., -1 , 0] = tau2
h_val =torch.stack((torch.zeros(len(a), device = device, dtype=torch.float32), p_max, p_pot, q_max, q_max, rho1, -rho2), 1)

In [45]:
G_val

tensor([[[-1.0000,  0.0000],
         [ 1.0000,  0.0000],
         [ 1.0000,  0.0000],
         ...,
         [ 0.0000,  1.0000],
         [-1.5000,  1.0000],
         [ 1.5000, -1.0000]],

        [[-1.0000,  0.0000],
         [ 1.0000,  0.0000],
         [ 1.0000,  0.0000],
         ...,
         [ 0.0000,  1.0000],
         [-1.5000,  1.0000],
         [ 1.5000, -1.0000]],

        [[-1.0000,  0.0000],
         [ 1.0000,  0.0000],
         [ 1.0000,  0.0000],
         ...,
         [ 0.0000,  1.0000],
         [-1.5000,  1.0000],
         [ 1.5000, -1.0000]],

        ...,

        [[-1.0000,  0.0000],
         [ 1.0000,  0.0000],
         [ 1.0000,  0.0000],
         ...,
         [ 0.0000,  1.0000],
         [-1.5000,  1.0000],
         [ 1.5000, -1.0000]],

        [[-1.0000,  0.0000],
         [ 1.0000,  0.0000],
         [ 1.0000,  0.0000],
         ...,
         [ 0.0000,  1.0000],
         [-1.5000,  1.0000],
         [ 1.5000, -1.0000]],

        [[-1.0000,  0.0000],
       

In [46]:
gt = generator_layer(a, G_val, h_val)[0]

In [47]:
model = KKT_NN.KKT_NN()
model.net.load_state_dict(torch.load("kkt_nn.pt", map_location = device))
model.net.to(device)

Net(
  (mlp): Sequential(
    (0): Linear(in_features=7, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): ResidualBlock(
      (linear): Linear(in_features=1024, out_features=1024, bias=True)
      (relu): LeakyReLU(negative_slope=0.01)
      (ln): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (3): ResidualBlock(
      (linear): Linear(in_features=1024, out_features=1024, bias=True)
      (relu): LeakyReLU(negative_slope=0.01)
      (ln): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (4): ResidualBlock(
      (linear): Linear(in_features=1024, out_features=1024, bias=True)
      (relu): LeakyReLU(negative_slope=0.01)
      (ln): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (5): Lin

In [48]:
pred = model.net(ds)[0]

In [68]:
torch.argmax((((gt - pred).abs()/(gt).abs()) * 100), 0)

tensor([334, 942])

In [69]:
gt[334]

tensor([0.0001, 0.0197])

In [74]:
ds[334]

tensor([3.2545e-01, 1.9734e-02, 6.8281e-05, 3.0000e-01, 3.0000e-01, 2.0000e-01,
        1.5000e-01])

In [70]:
pred[334]

tensor([0.0058, 0.0221], grad_fn=<SelectBackward0>)

In [67]:
(gt[584] - pred[584]).abs()

tensor([0.0021, 0.0443], grad_fn=<AbsBackward0>)